# Requirements

# Imports

In [1]:
import internetarchive
import os
import tarfile
import csv
from datetime import datetime
from tqdm.notebook import tqdm
import pandas as pd
import configparser
import mysql.connector
import json

In [2]:
import warnings

# Suppressing the warnings
warnings.filterwarnings('ignore') 

# Functions

## General

In [3]:
def check_folder(name_folder):            
    #Creating folder if that doesn't exist
    p = %pwd
    p = p + f'/{name_folder}'
    path = os.path.expanduser(p)
    if not os.path.exists(path):
        os.makedirs(path)
        print("{} created.".format(path))
        
    return path

In [4]:
def get_file_name(item_name, ext='*tar'):
    
    file_names = [f.name for f in internetarchive.get_files(item_name, glob_pattern= ext)]
    
    return file_names

In [5]:
def humanize(size_bytes):
    KB = 1 << 10
    MB = 1 << 20
    GB = 1 << 30

    if size_bytes < KB:
        return '{} B'.format(size_bytes)
    elif size_bytes < MB:
        return '{:.1f} KiB'.format(size_bytes/KB)
    elif size_bytes < GB:
        return '{:.1f} MiB'.format(size_bytes/MB)
    else:
        return '{:.1f} GiB'.format(size_bytes/GB)


## Cleaning and Transfomations

In [6]:
def get_data(file_path, file_name, word_bag = False, lang = False, db_update = False):
    
    #reading json
    df = pd.read_json(file_path, lines=True, compression='gzip')
    
    #updating total tweets
    if not update_db == False:
        update_db(table = 'control', field = f"count_total = '{len(df)}'", condition = f"name = '{file_name}'")
    
    #Filtering language if a language was sent
    if not lang == False:
        df = df[df.lang == lang]
    
    #taking away columns unnecessary
    df = df.loc[:, ['created_at', 'text', 'entities']]
    
    #filtering Tweets using subject word bag if a word bag was sent
    if not word_bag == False:
        bow = '|'.join(word_bag) # bag of word
        df = df[df['text'].str.contains(bow, case=False)]
        
        #updating filtered tweets
        if not update_db == False:
            update_db(table = 'control', field = f"count_filtered = '{len(df)}'", condition = f"name = '{file_name}'")
    
    return df

## Control Operations

### CSV

In [7]:
def control_op_csv(control_file):

    #if file doesn't exist I'll create it
    if not os.path.exists(control_file):
        with open(control_file, 'w', newline='') as control_csv:
            writer = csv.DictWriter(control_csv, fieldnames=['name','datetime'])
            writer.writeheader()
        control_csv.close()
        print("{} created.".format(control_file))
            
    #Reading the control file
    with open(control_file, 'r', newline='') as control_csv:
        reader = csv.DictReader(control_csv)
        reader_data = [r for r in reader]
        control_csv.seek(0)
    control_csv.close()
    
    return reader_data

In [8]:
def download_file_csv(item_name, file_name, download_folder = 'Downloads'):
    
    path = check_folder(download_folder)
    
    #calling function for getting control file
    control_path = path + '/control_download.csv'    
    reader_data = control_op_csv(control_path)
    
    #Getting Itens names
    item = internetarchive.get_item(item_name)
        
    #openning control file to add file names on the list
    with open(control_path, 'a', newline='') as control_download:
        writer = csv.DictWriter(control_download, fieldnames=['name','datetime'])
        
        #check if the file has been downloaded before
        if not any(row['name'] == file_name for row in reader_data):
            
            #downloading
            r = item.download(
                destdir=path,  # The directory to download files to
                ignore_existing=True,  # Skip files that already exist locally
                checksum=True,  # Skip files based on checksum
                verbose=True,  # Print progress to stdout
                retries=100,  # The number of times to retry on failed requests
                no_directory=True,  # Download withtout the identifier
                files = file_name)
            
            #Adding file name on control list
            row = {'name' : file_name, 'datetime': datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
            writer.writerow(row)
            
    control_download.close()
    
    path_file = path + f'/{file_name}'
    
    return path_file

In [9]:
def tar_file_csv(tar_file, file_name, extration_folder='Extraction'):

    path = check_folder(extration_folder)
    
    #calling function for getting control file
    control_path = path + '/control_extraction.csv'    
    reader_data = control_op(control_path)

    #openning control file to add file names on the list
    with open(control_path, 'a', newline='') as control_tar:
        writer = csv.DictWriter(control_tar, fieldnames=['name','datetime'])
        
        #check if the file has been extrated before
        if not any(row['name'] == file_name for row in reader_data):

            #Extracting file
            tar_file.extract(file_name, path=path)

            #Saving files name on control file
            row = {'name' : file_name, 'datetime': datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
            writer.writerow(row)
            
            path_file = path + f'/{file_name}'
            
        else:
            
            path_file = False
    
    control_tar.close()
    
    
    return path_file

### Mysql

#### Basic operations

In [10]:
def open_connection():

    #getting configuration
    config = configparser.ConfigParser()
    config.read('config.ini')

    # Connect to Mysql
    conn = mysql.connector.connect(
        host = config['mysql']['host'],
        user = config['mysql']['user'],
        password = config['mysql']['password'],
        database = config['mysql']['database']
    )
        
    return conn 

In [11]:
def update_db(table, field, condition):
    
    conn = open_connection()
    cursor = conn.cursor()

    cursor.execute(f"UPDATE {table} SET {field} WHERE {condition};")

    conn.commit()
    cursor.close()
    conn.close()

In [12]:
def query_db(query):
    
    conn = open_connection()
    cursor = conn.cursor()
    
    cursor.execute(query)

    conn.commit()
    cursor.close()
    conn.close()

In [13]:
def read_db(query):
    conn = open_connection()
    
    df = pd.read_sql_query(query, conn)
    
    conn.close()
    return df

#### Specific operations

In [14]:
def check_control_db(file_type, file_name = False):
    
    if file_name == False:
        
        check = read_db(f"SELECT name FROM control WHERE type ='{file_type}'").to_dict(orient='records')
    
    else:
        
        check = read_db(f"SELECT name FROM control WHERE type ='{file_type}' AND name = '{file_name}'").to_dict(orient='records')
        
        if len(check) > 0:
            check = True
        else:
            check = False

    return check

In [15]:
def download_file_db(item_name, file_name, download_folder = 'Downloads', type_control = 'download'):
    
    path = check_folder(download_folder)
    
    #query to get information from Mysql about control
    check = check_control_db(file_type = type_control, file_name = file_name)
                             
    #check if the file has been unzipped before
    if check == False:
    
        #getting Itens names
        item = internetarchive.get_item(item_name)
        
        print(f"Downloading {file_name}")

        #downloading
        r = item.download(
            destdir=path,  # The directory to download files to
            ignore_existing=True,  # Skip files that already exist locally
            checksum=True,  # Skip files based on checksum
            verbose=False,  # Print progress to stdout
            retries=600,  # The number of times to retry on failed requests
            no_directory=True,  # Download withtout the identifier
            files = file_name)
        
        #getting Metadata
        metadata = list(filter(lambda p: p["name"] == file_name, item.item_metadata['files']))

        #Adding file name on control list
        dt = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')
        sz = metadata[0]['size']
        query_db(f"INSERT INTO control (name, datetime, type, size) VALUES ('{file_name}', '{dt}', '{type_control}', '{sz}')")
        
        print(f"{file_name} downloaded")
    
        path_file = path + f'/{file_name}'
        
    else:
        
        path_file = False   
    
    return path_file

In [16]:
def tar_file_db(tar_file, file_name, extration_folder='Extraction', type_control = 'extraction'):
    
    path = check_folder(extration_folder)
    
    #query to get information from Mysql about control
    check = check_control_db(file_type = type_control, file_name = file_name)
    
    #check if the file has been unzipped before
    if check == False:    

        #Extracting file
        tar_file.extract(file_name, path=path)

        #Adding file name on control list
        dt = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')
        sz = tar_file.getmember(name=file_name).size
        query_db(f"INSERT INTO control (name, datetime, type, size) VALUES ('{file_name}', '{dt}', '{type_control}', '{sz}')")
        
        path_file = path + f'/{file_name}'
    
    else: #case the file has been unzipped
            
        path_file = False
        
    
    return path_file

# Code

In [17]:
import concurrent.futures

item_name = 'archiveteam-twitter-stream-2022-02'

#Getting Itens names
file_names = get_file_name(item_name = item_name)#, ext = '*jpg')

with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
    for dlf in tqdm(file_names, desc='Download progress'):

        #downloading the file
        file_future = executor.submit(download_file_db, item_name=item_name, file_name=dlf)

        if not file_future.result() == False:

            #Open tar file
            tar = tarfile.open(file_future.result(), "r")

            #Getting json files' names
            tar_file_names = list(filter(lambda t: t.endswith('.json.gz'), tar.getnames()))

            print(f"Unziping {dlf}")
            
            for tf in tqdm(tar_file_names, desc='Unzip progress'):

                #call funcition to extract files
                js_future = executor.submit(tar_file_db, tar_file=tar, file_name=tf)

                #if the file has already been unzipped that part will be skipped
                if not js_future.result() == False:

                    #Getting, cleaning and storing Data
                    bow = ['vaccination', 'vaccines', 'vaccine']
                    df = get_data(file_path = js_future.result(), file_name = tf, lang = 'en', word_bag = bow, db_update = True)

                    if not df.empty:

                        conn = open_connection()
                        cursor = conn.cursor()

                        for index, row in df.iterrows():
                            created_at = row['created_at']
                            text = row['text']
                            entities = json.dumps(row['entities'])

                            sql = "INSERT INTO tweets (created_at, text, entities) VALUES (%s, %s, %s)"
                            cursor.execute(sql, (created_at, text, entities))

                            conn.commit()

                        cursor.close()
                        conn.close()

                    #deleting json file after processing
                    os.remove(js_future.result())

            print(f"{dlf} Unziped")
            
            #closing tar file
            tar.close()

            #x =+ 1

            #removing file downloaded
            os.remove(file_future.result())

Download progress:   0%|          | 0/28 [00:00<?, ?it/s]

twitter-stream-20220201.tar downloaded
Unziping twitter-stream-20220201.tar


Unzip progress:   0%|          | 0/1440 [00:00<?, ?it/s]

twitter-stream-20220201.tar Unziped
twitter-stream-20220202.tar downloaded
Unziping twitter-stream-20220202.tar


Unzip progress:   0%|          | 0/1420 [00:00<?, ?it/s]

...

twitter-stream-20220226.tar Unziped
twitter-stream-20220227.tar downloaded
Unziping twitter-stream-20220227.tar


Unzip progress:   0%|          | 0/1440 [00:00<?, ?it/s]

twitter-stream-20220227.tar Unziped
twitter-stream-20220228.tar downloaded
Unziping twitter-stream-20220228.tar


Unzip progress:   0%|          | 0/1439 [00:00<?, ?it/s]

twitter-stream-20220228.tar Unziped


In [18]:
#ok